# Negative Prompting & Textual Inversion with Diffusers
This Colab shows how to:
1. Use **negative prompts** to steer *Stable Diffusion XL* away from unwanted artefacts.
2. Train a **textual inversion** concept embedding for a new style.


## 1. Install SDXL & diffusers (takes ~2 min on Colab T4)

In [ ]:
!pip -q install --upgrade diffusers accelerate transformers safetensors
import torch, random, os
from diffusers import StableDiffusionXLPipeline, TextualInversionLoaderMixin
pipe = StableDiffusionXLPipeline.from_pretrained('stabilityai/stable-diffusion-xl-base-1.0', torch_dtype=torch.float16, variant='fp16').to('cuda')


## 2. Basic generation

In [ ]:
prompt='A photo of a cute corgi wearing sunglasses, 4k' 
neg_prompt='bad anatomy, low resolution, blurry' 
image = pipe(prompt, negative_prompt=neg_prompt, num_inference_steps=30).images[0]
image

Notice how the negative prompt removes common SD artefacts.

## 3. Textual Inversion (tiny demo – 5 steps)

In [ ]:
# Load 4 example images of a new pattern (polka‑dot glass orb)
from diffusers import TextualInversionPipeline
ti_pipe = TextualInversionPipeline.from_pretrained(pipe)
images=['https://huggingface.co/datasets/diffusers/diffusers-images-docs/resolve/main/polka1.png',
        'https://huggingface.co/datasets/diffusers/diffusers-images-docs/resolve/main/polka2.png']
learned_embed, _ = ti_pipe.learn_new_concept(images, 'polka_orb', num_steps=5)

pipe.textual_inversion = learned_embed
pipe('A portrait of a cat holding a <polka_orb>', num_inference_steps=30).images[0]

### Exercise
1. Tweak `num_inference_steps`, `guidance_scale`.
2. Train your own concept on 5‑10 images and share in class.